# Importación de bibliotecas

In [ ]:
from openai import OpenAI
from getpass import getpass

# Creación de la función

## Instancia de OpenAI

In [ ]:
print("Insertar el API Key de OpenAI")
openai_key = getpass()

client = OpenAI(api_key=openai_key)


```
Se debe crear un sheet, con esta estructura 
```

Cabeceras del google sheet:
- Fecha
- Email
- Nombres
- Apellidos
- Documento
- Telefono

In [ ]:
# Datos del Google Sheet
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import datetime

# ID de tu hoja de cálculo de Google Sheets
# Lo encuentras en la URL:
# https://docs.google.com/spreadsheets/d/XXXXXXXXXXXXXXX/edit#gid=0
GOOGLE_SHEETS_SPREADSHEET_ID = "TU_ID_GOOGLE_SHEET"
# Nombre de la pestaña (hoja) dentro del archivo de Google Sheets
# Ejemplo: "Hoja1", "Ventas2025", etc.
GOOGLE_SHEETS_NAME = "TU_HOJA"
# Archivo de credenciales JSON de la cuenta de servicio
# Lo descargas desde Google Cloud Console al habilitar la API de Google Sheets
# Ejemplo: "credentials.json"
CREDENTIALS_FILE = "TU_ARCHIVO_CREDENTIALS.json"


Comporbar que llegamos al sheet

In [ ]:
def consultar_sheet(max_filas: int = 10):
    """
    Lee la fila de encabezados y hasta `max_filas` filas de datos de la hoja indicada.
    Devuelve un dict con 'headers' y 'rows' para inspección rápida.

    Args:
        max_filas: cantidad máxima de filas de datos (sin contar encabezado) a devolver.

    Returns:
        dict: {"headers": [...], "rows": [[...], [...], ...]}
    """
    try:
        scopes = [
            "https://www.googleapis.com/auth/spreadsheets.readonly",
            "https://www.googleapis.com/auth/drive.readonly",
        ]
        creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=scopes)
        service = build("sheets", "v4", credentials=creds)

        # Encabezados (primera fila)
        headers_resp = service.spreadsheets().values().get(
            spreadsheetId=GOOGLE_SHEETS_SPREADSHEET_ID,
            range=f"{GOOGLE_SHEETS_NAME}!1:1"
        ).execute()
        headers = headers_resp.get("values", [[]])
        headers = headers[0] if headers else []

        # Datos (primeras `max_filas` filas después del encabezado)
        data_resp = service.spreadsheets().values().get(
            spreadsheetId=GOOGLE_SHEETS_SPREADSHEET_ID,
            range=f"{GOOGLE_SHEETS_NAME}!A2:ZZ{max(2, max_filas+1)}"
        ).execute()
        rows = data_resp.get("values", [])

        return {"headers": headers, "rows": rows}

    except Exception as e:
        return {"headers": [], "rows": [], "error": str(e)}


## Función registro de cliente

In [ ]:
def registrar_cliente(email: str, nombres: str, apellidos: str, numero_documento: str, telefono: str):
    """
    Registra una nueva fila en Google Sheets

    Args:
        email: Correo electrónico del usuario
        nombres: Nombre del usuario
        apellidos: Apellido del usuario
        numero_documento: Número de documento
        telefono: Número de teléfono

    Returns:
        str: 'ok' si la operación fue exitosa
    """
    try:
        # Configuración de credenciales y parámetros de la hoja
        credentials_file = CREDENTIALS_FILE
        spreadsheet_id = GOOGLE_SHEETS_SPREADSHEET_ID
        sheet_name = GOOGLE_SHEETS_NAME

        # Scopes necesarios para acceder a Google Sheets y Drive
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive.file'
        ]

        # Autenticación con credenciales del servicio
        credentials = Credentials.from_service_account_file(credentials_file, scopes=scopes)
        service = build('sheets', 'v4', credentials=credentials)

        # Fecha y hora de registro (formato dd/mm/yyyy HH:MM:SS)
        fecha_registro = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

        # Preparar los datos de la nueva fila
        new_row = [
            fecha_registro,
            email,
            nombres,
            apellidos,
            numero_documento,
            telefono
        ]

        # Rango destino de la hoja (A:F → 6 columnas)
        range_to_append = f"{sheet_name}!A:F"

        # Cuerpo de la petición con los valores a insertar
        body = {
            'values': [new_row]
        }

        # Obtener datos existentes en la columna A para saber cuántas filas ocupadas hay
        existing_data = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!A:A"
        ).execute()

        # Calcular la próxima fila vacía (longitud de filas existentes + 1)
        existing_rows = len(existing_data.get('values', []))
        next_row = existing_rows + 1

        # Rango específico donde se insertará la nueva fila
        specific_range = f"{sheet_name}!A{next_row}:F{next_row}"

        # Insertar valores en la fila calculada
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=specific_range,
            valueInputOption='RAW',
            body=body
        ).execute()

        # Confirmación de filas agregadas (aunque en este caso siempre será 1 si es exitoso)
        rows_added = result.get("updates", {}).get("updatedRows", 0)

        print(f'Usuario registrado correctamente')
        print(f'Fecha de registro: {fecha_registro}')

        return 'ok'

    except Exception as error:
        # Captura de errores y relanzamiento para debugging
        print(f'Error al registrar el usuario: {error}')
        raise error


## Prueba de función nueva

In [ ]:
print(registrar_cliente("dlady@gmail.com", "Diana", "Laura", "25252545", "951561971"))

## Asignar función a asistente

In [ ]:
# Asistente
assistant_id = "asst_LRnpEfCeAI0capFlNzfIOIrZ"

In [ ]:
herramienta_registrar_cliente = {
    "type": "function",
    "function": {
        "name": "registrar_cliente",
        "description": "Registra un cliente cuando indica que quiere ser matricularse en un curso o programa.",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {
                    "type": "string",
                    "description": "Email del cliente."
                },
                "nombres": {
                    "type": "string",
                    "description": "Nombres del cliente."
                },
                "apellidos": {
                    "type": "string",
                    "description": "Apellidos del cliente."
                },
                "numero_documento": {
                    "type": "string",
                    "description": "Numero de documento del cliente"
                },
                "telefono": {
                    "type": "string",
                    "description": "Telefono del cliente.."
                }
            },
            "required": [
                "email",
                "nombres",
                "apellidos",
                "numero_documento",
                "telefono"
            ],
            "additionalProperties": False
        }
    }
}

In [ ]:
# Recuperar nuestro asistente
mi_asistente = client.beta.assistants.retrieve(assistant_id)
print(mi_asistente)

In [ ]:
# Obtener las tools que tenemos
herramientas_actuales = mi_asistente.tools or []
print(herramientas_actuales)
print(f"Herramientas existentes: {len(herramientas_actuales)}")

In [ ]:
# Agregar tu herramienta a las actuales
herramientas_actualizadas = herramientas_actuales + [herramienta_registrar_cliente]

# Actualizar el asistente con la nueva lista de herramientas
assistant_actualizado = client.beta.assistants.update(
    assistant_id=assistant_id,
    tools=herramientas_actualizadas
)

print(f"Asistente actualizado con {len(herramientas_actualizadas)} herramientas")

## Funcion de contar registros

In [ ]:
def contar_registros():
    """
    Cuenta cuántos registros (filas) hay en el Google Sheet

    Args:
        include_headers: Si incluir la fila de headers en el conteo (por defecto False)

    Returns:
        int: Número de registros
    """
    GOOGLE_SHEETS_SPREADSHEET_ID = ""
    GOOGLE_SHEETS_NAME = "registros"
    CREDENTIALS_FILE = ""
    include_headers = False
    try:
        # Scopes necesarios para Google Sheets
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive.file'
        ]

        # Cargar credenciales desde el archivo JSON
        credentials = Credentials.from_service_account_file(
            CREDENTIALS_FILE,
            scopes=scopes
        )

        # Crear el servicio de Google Sheets
        service = build('sheets', 'v4', credentials=credentials)

        # Obtener todas las filas con datos (columna A como referencia)
        result = service.spreadsheets().values().get(
            spreadsheetId=GOOGLE_SHEETS_SPREADSHEET_ID,
            range=f"{GOOGLE_SHEETS_NAME}!A:A"
        ).execute()

        # Contar filas con datos
        rows_with_data = result.get('values', [])
        total_rows = len(rows_with_data)

        if include_headers:
            registros = total_rows
            print(f'Total de filas (incluyendo headers): {registros}')
        else:
            # Restar 1 para excluir la fila de headers
            registros = max(0, total_rows - 1)
            print(f'Total de registros (sin headers): {registros}')
            print(f'Total de filas (con headers): {total_rows}')

        return f"Hay {registros} clientes registrados"

    except Exception as error:
        print(f'Error al contar registros: {error}')
        raise error

## Probar funcion

In [ ]:
print(contar_registros())

## Registrar herramienta de conteo de clientes

In [ ]:
herramienta_contar_registros = {
    "type": "function",
    "function": {
        "name": "contar_registros",
        "description": "Obtiene la cantidad de clientes registrados"
    }
}

In [ ]:
# Recuperar nuestro asistente
mi_asistente = client.beta.assistants.retrieve(assistant_id)
print(mi_asistente)

# Obtener las tools que tenemos
herramientas_actuales = mi_asistente.tools or []
print(herramientas_actuales)
print(f"Herramientas existentes: {len(herramientas_actuales)}")

# Agregar tu herramienta a las actuales
herramientas_actualizadas = herramientas_actuales + [herramienta_contar_registros]

# Actualizar el asistente con la nueva lista de herramientas
assistant_actualizado = client.beta.assistants.update(
    assistant_id=assistant_id,
    tools=herramientas_actualizadas
)

print(f"Asistente actualizado con {len(herramientas_actualizadas)} herramientas")

## Probar las funciones

In [ ]:
import json
import time

def chat_interactivo(assistant_id):
    """
    Chat interactivo con el asistente de OpenAI
    Escribe 'salir' para terminar la conversación
    """

    # Crear un nuevo thread para la conversación
    thread = client.beta.threads.create()
    print("🤖 ¡Hola! Soy tu asistente. Puedo ayudarte a contar clientes registrados y registrar nuevos clientes.")
    print("💡 Escribe 'salir' para terminar la conversación\n")

    while True:
        # Obtener entrada del usuario
        mensaje_usuario = input("👤 Tú: ").strip()

        # Verificar si quiere salir
        if mensaje_usuario.lower() in ['salir', 'exit', 'quit', 'bye']:
            print("🤖 ¡Hasta luego! 👋")
            break

        if not mensaje_usuario:
            print("🤖 Por favor escribe algo o 'salir' para terminar.")
            continue

        try:
            # Procesar mensaje y obtener respuesta
            respuesta = procesar_mensaje(assistant_id, thread.id, mensaje_usuario)
            print(f"🤖 Asistente: {respuesta}\n")

        except Exception as e:
            print(f"❌ Error: {str(e)}\n")

def procesar_mensaje(assistant_id, thread_id, mensaje_usuario):
    """
    Procesa un mensaje del usuario y maneja las llamadas a funciones
    """

    # Agregar mensaje del usuario al thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=mensaje_usuario
    )

    # Crear run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Manejar el run hasta completarse
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == "completed":
            break

        elif run_status.status == "requires_action":
            print("🔧 Ejecutando función...")

            # Obtener las llamadas a funciones requeridas
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name

                try:
                    if function_name == "contar_registros":
                        print("📊 Contando registros...")
                        resultado = contar_registros()
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": resultado
                        })

                    elif function_name == "registrar_cliente":
                        print("📝 Registrando nuevo cliente...")

                        # Parsear argumentos de la función
                        argumentos = json.loads(tool_call.function.arguments)

                        # Ejecutar la función con los argumentos
                        resultado = registrar_cliente(
                            email=argumentos.get('email'),
                            nombres=argumentos.get('nombres'),
                            apellidos=argumentos.get('apellidos'),
                            numero_documento=argumentos.get('numero_documento'),
                            telefono=argumentos.get('telefono')
                        )

                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": f"Cliente registrado exitosamente: {resultado}"
                        })

                    else:
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": f"Función no reconocida: {function_name}"
                        })

                except Exception as e:
                    print(f"❌ Error ejecutando {function_name}: {str(e)}")
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": f"Error al ejecutar la función: {str(e)}"
                    })

            # Enviar los resultados de vuelta a OpenAI
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run_status.status == "failed":
            return "❌ Lo siento, hubo un error procesando tu solicitud."

        elif run_status.status == "expired":
            return "⏰ La solicitud expiró. Intenta de nuevo."

        # Esperar un poco antes de verificar de nuevo
        time.sleep(1)

    # Obtener la respuesta final del asistente
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return messages.data[0].content[0].text.value

In [ ]:
chat_interactivo(assistant_id)

# Prueba en aplicacion web

In [ ]:
!pip install gradio

In [ ]:
# Instalar Gradio (ejecutar en la primera celda de Colab)
# !pip install gradio

import gradio as gr
import json
import time
from openai import OpenAI

# Configurar tu cliente OpenAI
client = OpenAI(api_key="TU_API_KEY")
assistant_id = assistant_id

# Variables globales para mantener el estado del chat
thread_id = None

def inicializar_chat():
    """Inicializa un nuevo thread para el chat"""
    global thread_id
    thread = client.beta.threads.create()
    thread_id = thread.id
    return "¡Hola! 👋 Soy tu asistente. Puedo ayudarte a contar clientes registrados y registrar nuevos clientes."

def procesar_mensaje_gradio(mensaje, historial):
    """
    Procesa un mensaje en Gradio y actualiza el historial
    """
    global thread_id

    if not thread_id:
        inicializar_chat()

    if not mensaje.strip():
        return historial, ""

    # Agregar mensaje del usuario al historial
    historial.append([mensaje, None])

    try:
        # Procesar con OpenAI
        respuesta = procesar_mensaje_openai(assistant_id, thread_id, mensaje)

        # Actualizar el historial con la respuesta
        historial[-1][1] = respuesta

    except Exception as e:
        historial[-1][1] = f"❌ Error: {str(e)}"

    return historial, ""

def procesar_mensaje_openai(assistant_id, thread_id, mensaje_usuario):
    """
    Procesa un mensaje con OpenAI y maneja las llamadas a funciones
    """

    # Agregar mensaje del usuario al thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=mensaje_usuario
    )

    # Crear run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Manejar el run hasta completarse
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == "completed":
            break

        elif run_status.status == "requires_action":
            # Obtener las llamadas a funciones requeridas
            tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name

                try:
                    if function_name == "contar_registros":
                        resultado = contar_registros()
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": resultado
                        })

                    elif function_name == "registrar_cliente":
                        # Parsear argumentos de la función
                        argumentos = json.loads(tool_call.function.arguments)

                        # Ejecutar la función con los argumentos
                        resultado = registrar_cliente(
                            email=argumentos.get('email'),
                            nombres=argumentos.get('nombres'),
                            apellidos=argumentos.get('apellidos'),
                            numero_documento=argumentos.get('numero_documento'),
                            telefono=argumentos.get('telefono')
                        )

                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": f"Cliente registrado exitosamente: {resultado}"
                        })

                    else:
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": f"Función no reconocida: {function_name}"
                        })

                except Exception as e:
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": f"Error al ejecutar la función: {str(e)}"
                    })

            # Enviar los resultados de vuelta a OpenAI
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run_status.status == "failed":
            return "❌ Lo siento, hubo un error procesando tu solicitud."

        elif run_status.status == "expired":
            return "⏰ La solicitud expiró. Intenta de nuevo."

        # Esperar un poco antes de verificar de nuevo
        time.sleep(1)

    # Obtener la respuesta final del asistente
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return messages.data[0].content[0].text.value

def nuevo_chat():
    """Reinicia el chat"""
    global thread_id
    thread_id = None
    mensaje_inicial = inicializar_chat()
    return [[None, mensaje_inicial]], ""

def crear_interfaz():
    """Crea la interfaz de Gradio"""

    # CSS personalizado para mejorar la apariencia
    css = """
    .container {
        max-width: 800px !important;
        margin: auto !important;
    }
    .chat-container {
        height: 500px !important;
    }
    .message {
        padding: 10px !important;
        margin: 5px !important;
        border-radius: 10px !important;
    }
    """

    with gr.Blocks(css=css, title="💬 Chat con Asistente de Clientes") as demo:

        gr.HTML("""
        <div style="text-align: center; padding: 20px;">
            <h1>💬 Chat con Asistente de Clientes</h1>
            <p>Tu asistente personal para gestionar clientes registrados</p>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=4):
                # Interfaz de chat
                chatbot = gr.Chatbot(
                    value=[[None, "¡Hola! 👋 Soy tu asistente. Puedo ayudarte a contar clientes registrados y registrar nuevos clientes."]],
                    elem_classes=["chat-container"],
                    bubble_full_width=False,
                    show_label=False
                )

                with gr.Row():
                    mensaje_input = gr.Textbox(
                        placeholder="Escribe tu mensaje aquí...",
                        show_label=False,
                        scale=4
                    )
                    enviar_btn = gr.Button("Enviar 📤", scale=1, variant="primary")

            with gr.Column(scale=1):
                gr.HTML("""
                <div style="padding: 20px; background: #f5f5f5; border-radius: 10px;">
                    <h3>💡 Comandos útiles:</h3>
                    <ul>
                        <li><b>Contar clientes:</b><br>"¿Cuántos clientes hay?"</li>
                        <li><b>Registrar cliente:</b><br>"Registra a Juan Pérez con email juan@gmail.com..."</li>
                    </ul>

                    <h3>📋 Datos necesarios para registro:</h3>
                    <ul>
                        <li>Nombres</li>
                        <li>Apellidos</li>
                        <li>Email</li>
                        <li>Documento</li>
                        <li>Teléfono</li>
                    </ul>
                </div>
                """)

                nuevo_chat_btn = gr.Button("🔄 Nuevo Chat", variant="secondary")

        # Eventos
        mensaje_input.submit(
            procesar_mensaje_gradio,
            [mensaje_input, chatbot],
            [chatbot, mensaje_input]
        )

        enviar_btn.click(
            procesar_mensaje_gradio,
            [mensaje_input, chatbot],
            [chatbot, mensaje_input]
        )

        nuevo_chat_btn.click(
            nuevo_chat,
            outputs=[chatbot, mensaje_input]
        )

    return demo

# Inicializar la aplicación
if __name__ == "__main__":
    # 2. Inicializar chat
    inicializar_chat()

    # 3. Crear y lanzar la interfaz
    demo = crear_interfaz()
    demo.launch(
        share=True,  # Crea una URL pública
        debug=True,
        server_name="0.0.0.0",
        server_port=7860
    )